In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('credit_card_transactions.csv')
df.info()
# df.describe()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 24 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   cc_num                 1296675 non-null  int64  
 3   merchant               1296675 non-null  object 
 4   category               1296675 non-null  object 
 5   amt                    1296675 non-null  float64
 6   first                  1296675 non-null  object 
 7   last                   1296675 non-null  object 
 8   gender                 1296675 non-null  object 
 9   street                 1296675 non-null  object 
 10  city                   1296675 non-null  object 
 11  state                  1296675 non-null  object 
 12  zip                    1296675 non-null  int64  
 13  lat                    1296675 non-null  float64
 14  long              

# Can we remove the credit card numbers?

### This dataset contains a credit card number associated with each transaction, but due to the random assignment by the card issuing authority, it's not likely that the credit card number itself can be very useful in determining whether or not a transaction is fraudulent. We can quickly scrub to confirm that all of the card numbers are valid using Luhn's algorithm. Note that since this dataset is notionally after the point of sale, it is expected that they are all valid card numbers. By confirming they are valid, we can remove them from out dataset since they are not going to be useful for other purposes.

### To quickly check if a credit card number is valid, you can use Luhn's algorithm: https://en.wikipedia.org/wiki/Luhn_algorithm
### A python module has been built to quickly check: https://pypi.org/project/luhncheck/

In [3]:
pip install luhncheck

  Using cached luhncheck-2.2.0-py3-none-any.whl (4.4 kB)
Note: you may need to restart the kernel to use updated packages.


In [4]:
# you may need to install luhncheck using pip
from luhncheck import is_luhn

# initialize two empty counters
check_counter = 0
error_counter = 0

# iterate through the credit card numbers and increment the check counter
for i in df['cc_num']:
    check_counter += 1
    # if the luhn check fails, the credit card number is invalid, and error increments
    if is_luhn(str(i)) == 'False':
        error_counter += 1

# print the results
print("Total count of credit card numbers: " + str(df['cc_num'].count()))
print("Values checked: " + str(check_counter))
print("Errors found: " + str(error_counter))

Total count of credit card numbers: 1296675
Values checked: 1296675
Errors found: 0


## No errors found indicates that all of these credit card numbers are valid. Again, this is not surprising, as presumably the merchant/payment processor are performing this check at the point of sale, effectively cutting out credit card number fraud before the transaction takes place. 

## Since all of the credit card numbers are valid, we will remove the field from the dataframe to reduce computational overhead.

In [5]:
print((df.memory_usage(deep=True).sum()/(1024*1024)))

1163.5949115753174


In [6]:
df.drop('cc_num', axis = 1, inplace = True)

In [7]:
print((df.memory_usage(deep=True).sum()/(1024*1024)))

1153.7020664215088


In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 23 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   merchant               1296675 non-null  object 
 3   category               1296675 non-null  object 
 4   amt                    1296675 non-null  float64
 5   first                  1296675 non-null  object 
 6   last                   1296675 non-null  object 
 7   gender                 1296675 non-null  object 
 8   street                 1296675 non-null  object 
 9   city                   1296675 non-null  object 
 10  state                  1296675 non-null  object 
 11  zip                    1296675 non-null  int64  
 12  lat                    1296675 non-null  float64
 13  long                   1296675 non-null  float64
 14  city_pop          

## We can see the total column count has decreased from 24 to 23, and we have reduced memory usage by ~10 MB

## What additional columns can we remove to reduce the load? 
#### Column 0 = an icrementing integer for each transaction that is not needed

Review whether the integer in column 0 or the unique transaction ID are more resource intesive, suspect transaction ID requires more bits in memory than integer so thats the one we should remove

In [10]:
print((df['Unnamed: 0'].memory_usage(deep=True)/(1024*1024)))
print((df['trans_num'].memory_usage(deep=True)/(1024*1024)))

9.892967224121094
110.0580244064331


## We can see in the output cell above that the memory usage for the first column titled 'Unnamed' compared to the memoryusage for the transaction number column. The Unnamed column includes a unique incrementing integer for each observation. The transaction number is a unique hexidecimal string for each observation. Since it is significantly longer, and uses a character space larger than the base 10 integer number system, it requires more than 10 times the memory to store.

## We will drop the transaction number column, and instead use the Unnamed column integer as our transaction itendifier.

In [11]:
df.drop('trans_num', axis = 1, inplace = True)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1296675 entries, 0 to 1296674
Data columns (total 22 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   Unnamed: 0             1296675 non-null  int64  
 1   trans_date_trans_time  1296675 non-null  object 
 2   merchant               1296675 non-null  object 
 3   category               1296675 non-null  object 
 4   amt                    1296675 non-null  float64
 5   first                  1296675 non-null  object 
 6   last                   1296675 non-null  object 
 7   gender                 1296675 non-null  object 
 8   street                 1296675 non-null  object 
 9   city                   1296675 non-null  object 
 10  state                  1296675 non-null  object 
 11  zip                    1296675 non-null  int64  
 12  lat                    1296675 non-null  float64
 13  long                   1296675 non-null  float64
 14  city_pop          

In [13]:
print((df.memory_usage(deep=True).sum()/(1024*1024)))

1043.6441640853882


## 110 MB of memory saved, reducing our total data size by ~9%

## OPTIONAL DATA CLEANING/FILLING ACTION: geocode missing merchant ZIPS

In [19]:
import plotly.express as px


dfhead = df.head(10000)

fig = px.scatter_geo(dfhead,lat='lat',lon='long', hover_name="trans_num")
fig.update_layout(title = 'World map', title_x=0.5)
fig.show()

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 0 entries
Data columns (total 23 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             0 non-null      int64  
 1   trans_date_trans_time  0 non-null      object 
 2   merchant               0 non-null      object 
 3   category               0 non-null      object 
 4   amt                    0 non-null      float64
 5   first                  0 non-null      object 
 6   last                   0 non-null      object 
 7   gender                 0 non-null      object 
 8   street                 0 non-null      object 
 9   city                   0 non-null      object 
 10  state                  0 non-null      object 
 11  zip                    0 non-null      int64  
 12  lat                    0 non-null      float64
 13  long                   0 non-null      float64
 14  city_pop               0 non-null      int64  
 15  job               